# TP1 - SQL

In [2]:
/*
-- Para comparar performance en los que hay más de una alternativa
SET STATISTICS IO ON
SET STATISTICS TIME ON;
*/

Commands completed successfully.

Total execution time: 00:00:00.010

***
**1. De la tabla [Person].[Contact], mostrar todos los datos de la tabla, ordenando por Apellido ascendente y nombre descendente.**

In [ ]:
SELECT *
FROM Person.Contact
ORDER BY LastName, FirstName DESC;

***
**2. De la tabla [Person].[Contact], muestre los nombres de las personas que comienzan con 'D' o que comienzan con "A", ordenado alfabéticamente.**

In [ ]:
SELECT DISTINCT pc.FirstName
FROM Person.Contact AS pc
WHERE pc.FirstName LIKE '[A/D]%'
ORDER BY 1;

--Alternativa más performante
SELECT pc.FirstName
FROM Person.Contact AS pc
WHERE pc.FirstName LIKE '[A/D]%'
GROUP BY pc.FirstName
ORDER BY 1;

***
**3 - De la tabla [Person].[Contact], muestra todos los registros que se modificaron después del 1/1/2013.**

In [ ]:
--Modifico año a 2003 porque las modificaciones van entre 1997 y 2005 y ordeno para evidenciar
SELECT *
FROM Person.Contact 
WHERE ModifiedDate > '2003-01-01'
ORDER BY ModifiedDate;

***
**4 - De la tabla [Person].[Address], muestre los nombres de las ciudades sin repetirlos.**

NOTA: Se puede hacer tb con group by. Diferencia de performance de DIstitnc con group by. EL group by es mas performante.

In [2]:
SELECT DISTINCT City 
FROM Person.Address;

--Alternativa más performante
SELECT City 
FROM Person.Address
GROUP BY City;

(575 rows affected)

(575 rows affected)

Displaying Top 5 rows.

Displaying Top 5 rows.

Total execution time: 00:00:00.096

City
Cheltenham
Kingsport
Suresnes
Baltimore
Reading


City
Cheltenham
Kingsport
Suresnes
Baltimore
Reading


***
**5 - Mostrar la cantidad de ventas que se realizaron con cada tipo de tarjetas de credito (se deben trabajar las tablas SalesOrderHeader y CreditCard).**

In [3]:
SELECT cc.CardType, COUNT(*) AS cantidad_ventas
FROM Sales.SalesOrderHeader soh
JOIN Sales.CreditCard cc
    ON  soh.CreditCardID = cc.CreditCardID
GROUP BY cc.CardType;

(4 rows affected)

Total execution time: 00:00:00.491

CardType,cantidad_ventas
SuperiorCard,7634
Vista,7432
Distinguish,7590
ColonialVoice,7678


In [4]:
-- Verifico que coincide con el total de ventas con tarjeta y que hay ventas sin tarjeta
SELECT COUNT(*) num_vtas_con_tarjeta
FROM  Sales.SalesOrderHeader
WHERE CreditCardID IS NOT NULL;

SELECT COUNT(*) num_vtas_sin_tarjeta
FROM  Sales.SalesOrderHeader
WHERE CreditCardID IS NULL;

SELECT COUNT(*) num_vtas_total
FROM  Sales.SalesOrderHeader;

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.066

num_vtas_con_tarjeta
30334


num_vtas_sin_tarjeta
1131


num_vtas_total
31465


***
**6 - De la tabla [Sales].[CreditCard], muestre todas las tarjetas con un año de vencimiento anterior al 2018 inclusive.**

In [5]:
-- Reemplazo 2018 por 2006 porque los vencimientos van entre 2005 y 2008
SELECT *
FROM Sales.CreditCard
WHERE ExpYear <= 2006;

(9550 rows affected)

Displaying Top 5 rows.

Total execution time: 00:00:00.275

CreditCardID,CardType,CardNumber,ExpMonth,ExpYear,ModifiedDate
1,SuperiorCard,33332664695310,11,2006,2003-08-30 00:00:00.000
2,Distinguish,55552127249722,8,2005,2004-01-06 00:00:00.000
3,ColonialVoice,77778344838353,7,2005,2004-02-15 00:00:00.000
4,ColonialVoice,77774915718248,7,2006,2003-06-21 00:00:00.000
5,Vista,11114404600042,4,2005,2003-03-05 00:00:00.000


***
**7 - Traer de la tabla Person.Contact los datos de contacto de todos los que NO son empleados (HumanResources.Employee).**

In [ ]:
SELECT pc.*
FROM Person.Contact pc
LEFT JOIN HumanResources.Employee hre
    ON pc.ContactID = hre.ContactID
WHERE hre.ContactID IS NULL;

--Alternativa menos performante (cheq)
SELECT * 
FROM Person.Contact pc
WHERE NOT EXISTS (SELECT 1
                  FROM HumanResources.Employee hre
                  WHERE pc.ContactID = hre.ContactID);

--Alternativa menos performante aún (cheq)
SELECT * 
FROM Person.Contact pc
WHERE pc.ContactID NOT IN (SELECT ContactID FROM HumanResources.Employee);

***
**8 - Crear una tabla [Test].[Tarjetas_vencidas] y cargar en la misma el resultado de la consulta 7.**

In [7]:
-- Asumo que se refiere a la consulta 6
-- Evito notación punto para no tener que crear una schema.
SELECT *
INTO Test_Tarjetas_vencidas 
FROM Sales.CreditCard   
WHERE ExpYear <= 2006;

(9550 rows affected)

Total execution time: 00:00:00.045

In [8]:
--Verifico
SELECT TOP(5) *
FROM Test_Tarjetas_vencidas;

(5 rows affected)

Total execution time: 00:00:00.027

CreditCardID,CardType,CardNumber,ExpMonth,ExpYear,ModifiedDate
13143,SuperiorCard,33333170223564,7,2005,2001-07-22 00:00:00.000
13144,Vista,11116273994765,12,2006,2003-09-21 00:00:00.000
13147,Vista,11119813587127,8,2005,2003-10-11 00:00:00.000
13149,Distinguish,55554406235985,11,2006,2003-10-10 00:00:00.000
13150,Vista,11115264873485,5,2005,2004-03-15 00:00:00.000


***
**9 - En la tabla [Test].[Tarjetas_vencidas] crear una nueva columna "bandera" con tipo int.**

In [9]:
-- Uso 0 como valor default
ALTER TABLE Test_Tarjetas_vencidas ADD bandera INT NOT NULL DEFAULT(0);

Commands completed successfully.

Total execution time: 00:00:00.022

In [10]:
--Verifico
SELECT TOP(5) *
FROM Test_Tarjetas_vencidas;

(5 rows affected)

Total execution time: 00:00:00.036

CreditCardID,CardType,CardNumber,ExpMonth,ExpYear,ModifiedDate,bandera
13143,SuperiorCard,33333170223564,7,2005,2001-07-22 00:00:00.000,0
13144,Vista,11116273994765,12,2006,2003-09-21 00:00:00.000,0
13147,Vista,11119813587127,8,2005,2003-10-11 00:00:00.000,0
13149,Distinguish,55554406235985,11,2006,2003-10-10 00:00:00.000,0
13150,Vista,11115264873485,5,2005,2004-03-15 00:00:00.000,0


***
**10 - Actualizar la tabla Test.Tarjetas_vencidas y dar valor 1 a la columna "bandera" cuando el vencimiento anterior del año 2015.**

In [11]:
--Elijo 2006 en lugar de 2015 porque los vencimientos van entre 2005 y 2008
UPDATE Test_Tarjetas_vencidas
SET bandera = 1
WHERE ExpYear < 2006;

(4743 rows affected)

Total execution time: 00:00:00.233

In [12]:
--Verifico
SELECT TOP(5) *
FROM Test_Tarjetas_vencidas
ORDER BY bandera DESC;

(5 rows affected)

Total execution time: 00:00:00.034

CreditCardID,CardType,CardNumber,ExpMonth,ExpYear,ModifiedDate,bandera
13147,Vista,11119813587127,8,2005,2003-10-11 00:00:00.000,1
13150,Vista,11115264873485,5,2005,2004-03-15 00:00:00.000,1
13143,SuperiorCard,33333170223564,7,2005,2001-07-22 00:00:00.000,1
13152,ColonialVoice,77775051853808,1,2005,2002-06-05 00:00:00.000,1
13155,ColonialVoice,77776381490516,9,2005,2004-04-29 00:00:00.000,1


In [13]:
--Para eliminarla
DROP TABLE Test_Tarjetas_vencidas;

Commands completed successfully.

Total execution time: 00:00:00.017

***
**11 - Crear una vista donde se muestre lo desarrollado en el punto 2.**

In [14]:
CREATE VIEW test_view
AS SELECT pc.FirstName
FROM Person.Contact AS pc
WHERE pc.FirstName LIKE '[A/D]%'
GROUP BY pc.FirstName;

Commands completed successfully.

Total execution time: 00:00:00.023

In [15]:
--Chequeo que está
SELECT TOP(5) *
FROM test_view;

(5 rows affected)

Total execution time: 00:00:00.059

FirstName
Amy
Anna
Daniel
Douglas
Alberto


In [ ]:
--Para eliminarla
DROP VIEW IF EXISTS test_view;

***
**12 - Crear una vista donde se muestren los datos de los empleados cuyo cumpleaños sea el dia de "hoy", Mostrando la fecha de nacimiento con el formato análogo a “15/11/2021”.**

In [21]:
CREATE VIEW birthday_view AS
SELECT CONVERT(varchar, hre.BirthDate, 103) AS BirthDate
FROM HumanResources.Employee hre
JOIN Person.Contact pc
      ON pc.ContactID = hre.ContactID
WHERE (MONTH(hre.BirthDate) = MONTH(GETDATE()) AND DAY(hre.BirthDate) = DAY(GETDATE())) OR
      (MONTH(hre.BirthDate) = 2 AND DAY(hre.BirthDate) = 29 AND MONTH(GETDATE()) = 3 AND DAY(GETDATE()) = 1)
/*
--Alternativa con FORMAT
SELECT FORMAT(hre.BirthDate, 'dd/MM/yyyy') AS BirthDate, pc.*
FROM HumanResources.Employee hre
JOIN Person.Contact pc
      ON pc.ContactID = hre.ContactID
WHERE (MONTH(hre.BirthDate) = MONTH(GETDATE()) AND DAY(hre.BirthDate) = DAY(GETDATE())) OR
      (FORMAT(hre.BirthDate, 'dd/MM') = '29/02' AND FORMAT(GETDATE(), 'dd/MM') = '01/03');
*/

Commands completed successfully.

Total execution time: 00:00:00.014

In [22]:
--Verifico
SELECT *
FROM birthday_view;

(3 rows affected)

Total execution time: 00:00:00.036

BirthDate
03/06/1977
03/06/1964
03/06/1951


In [26]:
--Para eliminarla
DROP VIEW IF EXISTS birthday_view;

Commands completed successfully.

Total execution time: 00:00:00.011

***
**12bis - A partir de la tabla SalesOrderHeader, Crear un Stored Procedure donde se muestren el dinero recaudado entre 2 fechas que se ingresaran por parametro.**


In [23]:
CREATE PROCEDURE GetRevenueTest
    @BeginDate date,   
    @EndDate date   
AS    
    SELECT SUM(TotalDue) ingresos
    FROM Sales.SalesOrderHeader
    WHERE OrderDate BETWEEN @BeginDate AND @EndDate;

Commands completed successfully.

Total execution time: 00:00:00.019

In [24]:
-- Ejecuto
EXECUTE GetRevenueTest @BeginDate = N'2002-03-01', @EndDate = N'2003-03-01';
-- O
--EXECUTE GetRevenueTest N'2002-03-01', N'2003-03-01';

(1 row affected)

Total execution time: 00:00:00.135

ingresos
43164261.7374


In [25]:
-- Para eliminarlo
DROP  PROCEDURE IF EXISTS  GetRevenueTest;

Commands completed successfully.

Total execution time: 00:00:00.009

***
**13 - Traer el nombre de las tablas pertenecientes al esquema Production.**

In [27]:
-- Incluyendo views
SELECT *
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'Production'
ORDER BY TABLE_NAME;

(28 rows affected)

Displaying Top 5 rows.

Total execution time: 00:00:00.078

TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
AdventureWorks,Production,BillOfMaterials,BASE TABLE
AdventureWorks,Production,Culture,BASE TABLE
AdventureWorks,Production,Document,BASE TABLE
AdventureWorks,Production,Illustration,BASE TABLE
AdventureWorks,Production,Location,BASE TABLE


In [28]:
-- Sin incluir views
SELECT *
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_SCHEMA = 'Production' AND TABLE_TYPE = 'BASE TABLE'
ORDER BY TABLE_NAME;

(25 rows affected)

Displaying Top 5 rows.

Total execution time: 00:00:00.055

TABLE_CATALOG,TABLE_SCHEMA,TABLE_NAME,TABLE_TYPE
AdventureWorks,Production,BillOfMaterials,BASE TABLE
AdventureWorks,Production,Culture,BASE TABLE
AdventureWorks,Production,Document,BASE TABLE
AdventureWorks,Production,Illustration,BASE TABLE
AdventureWorks,Production,Location,BASE TABLE


In [29]:
-- Alternativa 2 (query a sys.tables y usando function schema_name, no incluye views)
SELECT schema_name(schema_id) AS schema_name, name AS table_name
FROM sys.tables
WHERE schema_name(schema_id) = 'Production'
ORDER BY table_name;

(25 rows affected)

Displaying Top 5 rows.

Total execution time: 00:00:00.237

schema_name,table_name
Production,BillOfMaterials
Production,Culture
Production,Document
Production,Illustration
Production,Location


***
**14 - Traer por codigo la query utilizada para crear la vista correspondiente al punto 11.**

In [30]:
sp_helptext 'dbo.test_view';

Commands completed successfully.

Total execution time: 00:00:00.364

Text
CREATE VIEW test_view AS SELECT pc.FirstName FROM Person.Contact AS pc WHERE pc.FirstName LIKE '[A/D]%' GROUP BY pc.FirstName;


***
**15 - Traer por interfaz la query utilizada para crear la tabla Person.Contact.**

**En Azure Data Studio:(el usado)** Abriendo el panel de servers sobre la izquierda de la pantalla, desplegando el server actual, el directorio `Tables`, haciendo click derecho sobre `Person.Contact` y seleccionando `Script as Create` me abre un archivo `.sql` con el `CREATE` statement correspondiente a esa tabla.

**En SQL Server Management Studio:** Acá una [explicación con las imágenes de la GUI](https://docs.microsoft.com/en-us/sql/ssms/scripting/generate-scripts-sql-server-management-studio?view=sql-server-ver16).

***
**16 - Traer por Codigo la query utilizada para crear la tabla Person.Contact.**

Entiendo que no hay análogo a la herramienta sp_helptext [(doc)](https://docs.microsoft.com/en-us/sql/relational-databases/system-stored-procedures/sp-helptext-transact-sql?view=sql-server-ver16) para tablas, y que hay algunos objetos de SQL Server para los cuales se debe obtener el script vía T-SQL o armando algo específico con SQL Server Management Objects Framework (aka SMO).

Armando el script de T_SQL `recreate_creation_script.sql` y se puede correr el comando...
```
sqlcmd -U SA -d "AdventureWorks" -i recreate_creation_script.sql -o creation_script.sql
```
...y obtener el `creation_script.sql`.